This idea of this notebook is to create a first (simple) DL model using all the same features used in the multinomial logit model.

Much inspiration was derived from https://towardsdatascience.com/use-machine-learning-to-predict-horse-racing-4f1111fb6ced.

In [43]:
import math
from importlib import reload
import deeplearninglib
reload(deeplearninglib)
from deeplearninglib import *

import wandb

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


In [44]:
# read in data

# select the same features used in the multinomial model
# X_columns = ["age", "sire_sr", "dam_sr", "trainer_sr", "daysLTO", "position1_1", "position1_2", "position1_3", "position1_4", "position2_1", "position2_2", "position2_3", "position2_4", "position3_1", "position3_2", "position3_3", "position3_4", "entire", "gelding", "blinkers", "visor", "cheekpieces", "tonguetie"]
X_columns = ["mkt_prob"]
y_columns = ["win"]

train_data = RacesDataset('data\\runners_train.csv', X_columns, y_columns)
test_data = RacesDataset('data\\runners_test.csv', X_columns, y_columns)

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [45]:
pd.options.display.max_columns = 1000 # was 20
train_data.races.iloc[:, train_data.X_columns].head()

mkt_prob                                                    \
stall_number        1         2         3         4         5         6    
race_id                                                                    
11504         0.100000  0.038462  0.125000  0.058824  0.142857  0.053763   
11505         0.019608  0.029412  0.142857  0.038462  0.400000  0.058824   
11506         0.266667  0.038462  0.014925  0.058824  0.181818  0.038462   
11507         0.076923  0.058824  0.019608  0.222222  0.444444  0.029412   
11508         0.038462  0.038462  0.133333  0.047619  0.230947  0.200000   

                                                                               \
stall_number        7         8         9         10        11   12   13   14   
race_id                                                                         
11504         0.090909  0.125000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
11505         0.053763  0.142857  0.029412  0.222222  0.153846  0.0  0.0  0.0   
11506         0.111111  0.029412  0.111111  0.307692  0.000000  0.0  0.0  0.0   
11507         0.125000  0.153846  0.066667  0.000000  0.000000  0.0  0.0  0.0   
11508         0.058824  0.181818  0.250000  0.000000  0.000000  0.0  0.0  0.0   

                        
stall_number   15   16  
race_id                 
11504         0.0  0.0  
11505         0.0  0.0  
11506         0.0  0.0  
11507         0.0  0.0  
11508         0.0  0.0

In [46]:
# build the neural network

output_layer_nodes = train_data.y.shape[1]
input_layer_nodes = train_data.X.shape[1]

torch.manual_seed(0)
model = LRLS(input_layer_nodes, output_layer_nodes).to(device) # linear-relu-linear-softwax neural net with 1 hidden layer
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param} \n")

Model structure: LRLS(
  (neural_network): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=False)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=False)
    (3): Softmax(dim=1)
  )
)


Layer: neural_network.0.weight | Size: torch.Size([16, 16]) | Values : Parameter containing:
tensor([[-0.0019,  0.1341, -0.2058, -0.1840, -0.0963,  0.0670, -0.0050,  0.1982,
         -0.0222,  0.0662, -0.0756, -0.0491, -0.2388, -0.1656, -0.1031,  0.0093],
        [ 0.0988,  0.1500, -0.1695, -0.1089,  0.0908,  0.2076, -0.0515,  0.1871,
         -0.0403,  0.0265,  0.2264, -0.2319, -0.1574, -0.0633, -0.0974,  0.2160],
        [-0.1620, -0.1151, -0.1747, -0.2341, -0.1459,  0.2149,  0.1116,  0.1212,
          0.0131, -0.1282,  0.0423, -0.2334, -0.1806, -0.1289,  0.1577,  0.1466],
        [-0.1109, -0.0090,  0.1599,  0.2485,  0.0992,  0.0338,  0.1676, -0.1472,
          0.0466, -0.1938, -0.1733, -0.1291,  0.1131,  0.1005, -0.1481,  0.0755],
        [ 0.1372, -0.0316,  0.

In [47]:
# example to show how model is used from prediction

X = torch.rand(1, input_layer_nodes, device=device)
logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
y_pred = logits.argmax(1)
print(f"Predicted class: {y_pred}")
print(X.shape)

Predicted class: tensor([1])
torch.Size([1, 16])


In [48]:
%env "WANDB_NOTEBOOK_NAME" "3b_Deep Learning.ipynb"

# optimizing model parameters

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="horse-racing-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 10e-1,
    "architecture": "Simplest",
    "dataset": "full",
    "epochs": 100,
    "device": device
    }
)

# hyperparameters
learning_rate = wandb.config.learning_rate
epochs = wandb.config.epochs

# initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    (acc, loss) = test_loop(test_dataloader, model, loss_fn, device)
    wandb.log({"acc": acc, "loss": loss})
print("Done!")

# finish the wandb run, necessary in notebooks
wandb.finish()

env: "WANDB_NOTEBOOK_NAME"="3b_Deep Learning.ipynb"


Epoch 1
-------------------------------
loss: 2.772529  [   64/16620]
loss: 2.771539  [ 6464/16620]
loss: 2.814276  [12864/16620]
Test Error: 
 Accuracy: 12.5%, Avg loss: 2.777272 

Epoch 2
-------------------------------
loss: 2.770969  [   64/16620]
loss: 2.766967  [ 6464/16620]
loss: 2.806212  [12864/16620]
Test Error: 
 Accuracy: 11.8%, Avg loss: 2.766496 

Epoch 3
-------------------------------
loss: 2.763449  [   64/16620]
loss: 2.743095  [ 6464/16620]
loss: 2.773904  [12864/16620]
Test Error: 
 Accuracy: 10.8%, Avg loss: 2.738477 

Epoch 4
-------------------------------
loss: 2.741557  [   64/16620]
loss: 2.699145  [ 6464/16620]
loss: 2.738662  [12864/16620]
Test Error: 
 Accuracy: 18.2%, Avg loss: 2.714659 

Epoch 5
-------------------------------
loss: 2.714838  [   64/16620]
loss: 2.665002  [ 6464/16620]
loss: 2.680048  [12864/16620]
Test Error: 
 Accuracy: 24.0%, Avg loss: 2.669797 

Epoch 6
-------------------------------
loss: 2.670493  [   64/16620]
loss: 2.617899  [ 64

acc,▂▁▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
loss,█▇▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.31937
loss,2.55807


In [49]:
for para_name, para_vals in model.named_parameters():
    np.savetxt(para_name + ".csv", para_vals.data.numpy(), fmt='%6.3f', delimiter=",")